<a href="https://colab.research.google.com/github/Bhaanavee/SkillChase/blob/main/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

df = pd.read_csv("StudentsPerformance.csv")
df.head()


,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [4]:
# CREATING SQL DATABASE
import sqlite3
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# CREATE A TABLE IN THE DATABASE
df.to_sql('students', conn, index=False, if_exists='replace')
# VERIFYING RECORDS
cursor.execute('''SELECT * FROM students LIMIT 5''')
rows = cursor.fetchall()
for row in rows:
    print(row)



('female', 'group B', "bachelor's degree", 'standard', 'none', 72, 72, 74)
('female', 'group C', 'some college', 'standard', 'completed', 69, 90, 88)
('female', 'group B', "master's degree", 'standard', 'none', 90, 95, 93)
('male', 'group A', "associate's degree", 'free/reduced', 'none', 47, 57, 44)
('male', 'group C', 'some college', 'standard', 'none', 76, 78, 75)


In [5]:
query = '''
SELECT "race/ethnicity", AVG("math score") AS avg_math_score, AVG("reading score") AS avg_reading_score, AVG("writing score") AS avg_writing_score
FROM students
GROUP BY "race/ethnicity"
'''
scores_by_race = pd.read_sql(query, conn)
scores_by_race


,race/ethnicity,avg_math_score,avg_reading_score,avg_writing_score
0,group A,61.629213,64.674157,62.674157
1,group B,63.452632,67.352632,65.600000
2,group C,64.463950,69.103448,67.827586
3,group D,67.362595,70.030534,70.145038
4,group E,73.821429,73.028571,71.407143


In [6]:
query = '''
SELECT "parental level of education", COUNT(*) AS student_count
FROM students
GROUP BY "parental level of education"
'''
parental_education_count = pd.read_sql(query, conn)
parental_education_count


,parental level of education,student_count
0,associate's degree,222
1,bachelor's degree,118
2,high school,196
3,master's degree,59
4,some college,226
5,some high school,179


In [7]:
query = '''
SELECT lunch, AVG("math score") AS avg_math_score, AVG("reading score") AS avg_reading_score, AVG("writing score") AS avg_writing_score
FROM students
GROUP BY lunch
'''
lunch_comparison = pd.read_sql(query, conn)
lunch_comparison


,lunch,avg_math_score,avg_reading_score,avg_writing_score
0,free/reduced,58.921127,64.653521,63.022535
1,standard,70.034109,71.654264,70.823256


In [8]:
# ADD COLUMN FOR AVG SCORE
cursor.execute('''
    ALTER TABLE students ADD COLUMN average_score REAL
''')
cursor.execute('''
    UPDATE students
    SET average_score = ("math score" + "reading score" + "writing score") / 3
''')

# The top 10% performers
query = '''
SELECT * FROM students
ORDER BY average_score DESC
LIMIT (SELECT COUNT(*) FROM students) * 0.1
'''
top_performers = pd.read_sql(query, conn)
top_performers


,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,average_score
0,female,group E,bachelor's degree,standard,none,100,100,100,100.0
1,male,group E,bachelor's degree,standard,completed,100,100,100,100.0
2,female,group E,associate's degree,standard,none,100,100,100,100.0
3,female,group E,bachelor's degree,standard,completed,99,100,100,99.0
4,female,group D,some high school,standard,completed,97,100,100,99.0
...,...,...,...,...,...,...,...,...,...
95,female,group C,associate's degree,standard,completed,83,85,90,86.0
96,female,group C,bachelor's degree,standard,none,81,88,90,86.0
97,male,group D,master's degree,standard,none,95,81,84,86.0
98,female,group D,some high school,standard,completed,80,92,88,86.0
